# --- Day 8: Space Image Format ---
The Elves' spirits are lifted when they realize you have an opportunity to reboot one of their Mars rovers, and so they are curious if you would spend a brief sojourn on Mars. You land your ship near the rover.

When you reach the rover, you discover that it's already in the process of rebooting! It's just waiting for someone to enter a BIOS password. The Elf responsible for the rover takes a picture of the password (your puzzle input) and sends it to you via the Digital Sending Network.

Unfortunately, images sent via the Digital Sending Network aren't encoded with any normal encoding; instead, they're encoded in a special Space Image Format. None of the Elves seem to remember why this is the case. They send you the instructions to decode it.

Images are sent as a series of digits that each represent the color of a single pixel. The digits fill each row of the image left-to-right, then move downward to the next row, filling rows top-to-bottom until every pixel of the image is filled.

Each image actually consists of a series of identically-sized layers that are filled in this way. So, the first digit corresponds to the top-left pixel of the first layer, the second digit corresponds to the pixel to the right of that on the same layer, and so on until the last digit, which corresponds to the bottom-right pixel of the last layer.

In [21]:
import numpy as np
from collections import Counter

width = 25
height = 6

test1 = "123456789012"

def str_to_array(pixels, width, height):
    ar = np.array([int(x) for x in pixels])
    layers = len(ar) / (width * height)
    return ar.reshape([-1, height, width])

str_to_array(test1, 3, 2)

array([[[1, 2, 3],
        [4, 5, 6]],

       [[7, 8, 9],
        [0, 1, 2]]])

In [14]:
with open('../data/08a.txt', 'r') as f:
    pixels = f.read().strip()
len(pixels)

15000

In [37]:
layered_pixels = str_to_array(pixels, width, height)

counts = {}
for i, layer in enumerate(layered_pixels):
    c = Counter(layer.reshape(-1,))
    counts[i] = c

layer_fewest_zeros = min(counts, key=lambda x: counts[x][0])
counts[layer_fewest_zeros]

Counter({2: 129, 1: 16, 0: 5})

In [35]:
counts[11][1] * counts[11][2]

2064

--- Part Two ---
Now you're ready to decode the image. The image is rendered by stacking the layers and aligning the pixels with the same positions in each layer. The digits indicate the color of the corresponding pixel: 0 is black, 1 is white, and 2 is transparent.

The layers are rendered with the first layer in front and the last layer in back. So, if a given position has a transparent pixel in the first and second layers, a black pixel in the third layer, and a white pixel in the fourth layer, the final image would have a black pixel at that position.

For example, given an image 2 pixels wide and 2 pixels tall, the image data 0222112222120000 corresponds to the following image layers:
```
Layer 1: 02
         22

Layer 2: 11
         22

Layer 3: 22
         12

Layer 4: 00
         00
```
Then, the full image can be found by determining the top visible pixel in each position:

The top-left pixel is black because the top layer is 0.
The top-right pixel is white because the top layer is 2 (transparent), but the second layer is 1.
The bottom-left pixel is white because the top two layers are 2, but the third layer is 1.
The bottom-right pixel is black because the only visible pixel in that position is 0 (from layer 4).
So, the final image looks like this:
```
01
10
```
What message is produced after decoding your image?

In [208]:
h, l, w = layered_pixels.shape
h, l, w

(100, 6, 25)

In [211]:
shaped_layered = layered_pixels.reshape(1, -1, l*w)[0]
shaped_layered.shape

(100, 150)

In [214]:
def combine_layers(shaped, verbose=False):
    """This is very NON-ideal. I'm sure there's a way to slice w/ 
    NumPy magic and get something that avoids nested for loops."""
    output = np.array([2 for _ in range(shaped.shape[1])])
    
    if verbose:
        print(output.shape, 'output')
        print(shaped.shape, 'shape of input')
    
    for layer in shaped[::-1]:
        new_output = []
        
        for new, old in zip(layer, output):
            if new != 2:
                new_output.append(new)
            else: 
                new_output.append(old)

        output = np.array(new_output)
        
    if verbose:    
        print(output.shape, 'shape of output')
        
    return output

In [215]:
test2 = str_to_array('0222112222120000', 2, 2)

shaped = test2.reshape(1,-1,4)[0]
#print(shaped.shape, '  is the shape')

assert combine_layers(shaped).tolist() == [0,1,1,0]
print('test passed')

test passed


In [227]:
combined = combine_layers(shaped_layered)
finished = combined.reshape(height, width)
print(finished)

[[1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0]
 [1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0]
 [1 1 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 0]
 [1 0 1 0 0 1 1 1 1 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1 0]
 [1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0]
 [1 0 0 1 0 1 0 0 1 0 0 1 1 0 0 1 1 1 1 0 1 0 0 1 0]]
